In [1]:
# The ultimate target feature: time from one stop to another

# Imports
import pandas as pd
import numpy as np
from datetime import date, datetime
from patsy import dmatrices
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import export_graphviz, DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from statsmodels.formula.api import ols

# Read csv file into a dataframe.
df = pd.read_csv('csv_data/bus_route4_clean.csv')

# 1.0 Model Analysis

## 1.1 Random Forest Regression

In [2]:
y, X = dmatrices('time_diff_to_destination ~ distance_to_end + Temperature + Humidity + HourOfDay +IsWeekend + SchoolHoliday + Vehicle_Journey_ID + Start_date', df, return_type="dataframe") 
y = np.ravel(y)
X.head(5)

,Intercept,Start_date[T.2013-01-02],Start_date[T.2013-01-03],Start_date[T.2013-01-04],Start_date[T.2013-01-05],Start_date[T.2013-01-06],Start_date[T.2013-01-07],Start_date[T.2013-01-08],Start_date[T.2013-01-09],Start_date[T.2013-01-10],...,Start_date[T.2013-01-29],Start_date[T.2013-01-30],Start_date[T.2013-01-31],distance_to_end,Temperature,Humidity,HourOfDay,IsWeekend,SchoolHoliday,Vehicle_Journey_ID
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,46.0,2.0,89.0,8.0,0.0,1.0,15475.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,45.0,2.0,89.0,8.0,0.0,1.0,15475.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,44.0,2.0,89.0,8.0,0.0,1.0,15475.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,43.0,2.0,89.0,8.0,0.0,1.0,15475.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,42.0,2.0,89.0,8.0,0.0,1.0,15475.0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123) 

In [4]:
scaler = preprocessing.StandardScaler().fit(X_train)

In [5]:
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=10))

In [6]:
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

In [ ]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
r2_score(y_test, pred)

Good, closer to 1 the better

In [ ]:
mean_squared_error(y_test, pred)

Good, closer to 0 the better.

This saves the model for later.

In [ ]:
joblib.dump(clf, '../flask_app/static/rf_regressor.pkl')

In [ ]:
X_test.head()

This can then be reloaded to be used in the flask app. As seen below.

In [ ]:
clf2 = joblib.load('../flask_app/static/rf_regressor.pkl')

In [ ]:
clf2.predict(X_test)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)

In [ ]:
df.shape

## To Do

Date is in the model as a variable making significant impact as cannot group random forest by dates - to confirm why it makes an impact (improves score about 7% but we should  understand is this equiv to groupby or coincidence or a proxy for events).

Getting a memory error with feature importance so for now using the linear results to guide - issue with rain: clearly important - we know that - and humidity is important, but no rain columns are making any impact. See commented out code below.

In [ ]:
#rfc.fit(X_train, y_train)

In [ ]:
#pd.DataFrame({'feature': X_train.columns, 'importance':rfc.feature_importances_})